#### Dataframe Joins and Optimizations
SparkSQL builds in several optimizations while performing joins and other operations on dataframes.

We can check out the execution plan of SparkSQL for any operation using the function "explain()"

When one of the dataframes in the join operation is small enough to fit into a workernode then Spark SQL plans to "broadcast" this dataframe. This is generally referred to as BroadcashJoin. When a dataframe is "broadcasted" then a copy of it is sent to all worker nodes. This avoids shuffling of data when joining it with the larger dataframe as the smaller dataframe is compltely available locally for each partition of the larger dataframe.

The above optimization is done automatically by SparkSQL's built-in optimizer. However we can also check to ensure it is done and in case it is not done it can be done manually also as shown in the example below.

The setting "spark.sql.autoBroadcastJoinThreshold" determines whether a dataframe is small enough to be broadcast or not. The default of this setting is 10 MB (10485760). We can override this when we know that the smaller dataframe is just above this threshold and can be managed to be fit into each worker node. 

In [ ]:
import findspark

In [ ]:
findspark.init()

In [ ]:
# Now import pyspark. And start SparkSession
import pyspark

In [ ]:
# The following three lines are required if you are running PySpark using Jupyter notebook with Python3 as kernel 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark SQL Functions - Examples").getOrCreate()

Create DataFrame from data source - csv file

In [ ]:
customerDF = spark.read.load("customers.tsv", format="csv", sep="\t", inferSchema="true")
salestxnDF = spark.read.load("salestxns.tsv", format="csv", sep="\t", inferSchema="true")

In [ ]:
customerDF.printSchema()

In [ ]:
salestxnDF.printSchema()

In [ ]:
new_names_cDF = ['customer_id','customer_name','customer_city','customer_state','customer_zipcode']

In [ ]:
new_names_sDF = ['salestxn_id','category_id','category_name','product_id','product_name','product_price','product_quantity','customer_id']

In [ ]:
cDF2=customerDF.toDF(*new_names_cDF)

In [ ]:
sDF2=salestxnDF.toDF(*new_names_sDF)

In [ ]:
cDF2.printSchema()

In [ ]:
cDF2.show(5)

In [ ]:
cDF2.count()

In [ ]:
sDF2.printSchema()

In [ ]:
sDF2.show(5)

In [ ]:
cDF2.rdd.getNumPartitions()

In [ ]:
sDF2.count()

In [ ]:
sDF2.rdd.getNumPartitions()

Join the two dataframes on customer_id column

In [ ]:
jDF1=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [ ]:
jDF1.printSchema()

In [ ]:
jDF1.show(5)

Use explain() function to see the type of join used by Spark SQL

In [ ]:
jDF1.explain()

Using explain(True) function gives the outline of the initial logical plans besides the final physical plan used for executing the join operation

In [ ]:
jDF1.explain(True)

In [ ]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

By setting this value to -1 broadcasting can be disabled.

In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

In [ ]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

In [ ]:
jDF2=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [ ]:
# Since autoBroadcastJoinThreshold is disabled by setting it to -1, physical plan shows BroadCastHashJoin it is not used.
jDF2.explain()

We can notice that broadcast join is not used now as the threshold is disabled.

In [ ]:
jDF2.printSchema()

In [ ]:
jDF2.show(5)

In [ ]:
from pyspark.sql import functions as F

The lines below show how to mark a dataframe for broadcast in case required to be done manually.

In [ ]:
jDF3=cDF2.hint("broadcast").join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [ ]:
jDF3.explain()

In [ ]:
jDF3.printSchema()

In [ ]:
jDF3.show(5)

In [ ]:
jDF4=F.broadcast(cDF2).join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [ ]:
jDF4.explain()

In [ ]:
jDF4.printSchema()

In [ ]:
jDF4.show(5)

In [ ]:
cDF2.cache()

In [ ]:
cDF2.count()

In [ ]:
sDF2.persist()

sDF2.count()

In [ ]:
jDF5=F.broadcast(cDF2).join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [ ]:
jDF5.explain()

# In the physical plan below we can see "InMemoryTableScan" being used for both the dataframes
# which was not the case in the previous one.
# This means that the data is read from the cache() insted of re-building the dataframes from scratch.

In [ ]:
# In the statement below, after join we are doing a filter and also we are doing select of only 5 columns.

jDF6=cDF2.hint("broadcast").join(sDF2,cDF2.customer_id==sDF2.customer_id).\
select('customer_name','customer_city', 'customer_state', 'product_name','product_price','product_quantity').where(cDF2.customer_state=='IL')

# In the physical plan shown after the explain statement, we can see projection (select clause) push and filter push.
# This means that the query execution engine executes projection and then filter and only after that join is executed,
# which is more optimal way of executing the whole statement.

In [ ]:
jDF6.explain()

In [ ]:
spark.stop()